<a href="https://colab.research.google.com/github/FARTASH-421/AI2/blob/master/Project_CR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
def mul(ls, num):
  num = int(num)
  if num == 0:
      return ''.join([str(0) for i in range(len(ls))])
  else:
      return ls

In [256]:
def sum(ls_num, num):
  bin_to_Int = int (ls_num, 2)
  bin_to_Int += int(num, 2)
  bin_to_Int = bin_to_Int % 65535
  lis = format(bin_to_Int, 'b')
  l = len(lis)
  len_zero = 16 - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front+lis


In [161]:
def xor(lis_1, lis_2):
  int_x = int(lis_1,2)
  int_y = int(lis_2, 2)
  result = int_x ^ int_y
  str_result = format(result, 'b')
  l = len(str_result)
  len_zero = 16 - l
  zero_front = ''.join([str(0) for i in range(len_zero)])
  return zero_front + str_result

In [220]:
def Round (list_P, list_K):
    P1 = list_P[0 :16]
    P2 = list_P[16:32]
    P3 = list_P[32:48]
    P4 = list_P[48:64]

    # print(f'P1: size:{len(P1)} is-> {P1}')
    # print(f'P2: size:{len(P2)} is-> {P2}')
    # print(f'P3: size:{len(P3)} is-> {P3}')
    # print(f'P4: size:{len(P4)} is-> {P4}')

    step_1 = mul (P1, list_K[0])      #1010101111111111
    step_2 = sum (P2, list_K[1])      #1110000011111000
    step_3 = sum (P3, list_K[2])      #0000111100101010
    step_4 = sum (P4, list_K[3])      #1010110101010111
    step_5 = xor (step_1, step_3)     #1010010011010101
    step_6 = xor (step_2, step_4)     #0100110110101111
    step_7 = mul (step_5, list_K[4])  #0000000000000000
    step_8 = sum (step_6, step_7)     #0100110110101111
    step_9 = mul (step_8, list_K[5])  #0100110110101111
    step_10 = sum (step_7, step_9)    #0100110110101111

    step_11 = xor (step_1, step_9)    #1110011001010000
    step_12 = xor (step_3, step_9)    #0100001010000101
    step_13 = xor (step_2, step_10)   #1010110101010111
    step_14 = xor (step_4, step_10)   #1110000011111000
    return step_11 + step_12 + step_13 + step_14



In [234]:
def Block_Cryptography(plaintext, key):
  index_key = 0
  for i in range(8):
    part_key = key[index_key:(index_key+6)]
    plaintext = Round(plaintext,key)
    index_key += 6

  R_1 = plaintext[0:16]
  R_2 = plaintext[16:32]
  R_3 = plaintext[32:48]
  R_4 = plaintext[48:64]


  step_1 = mul (R_1, key[index_key])
  step_2 = sum (R_2, key[index_key+1])
  step_3 = sum (R_3, key[index_key+2])
  step_4 = mul (R_4, key[index_key+3])

  ciphertext = step_1 + step_2 +step_3 + step_4
  return ciphertext


In [237]:
# ...... Test def Block_Cryptography .............

ls = '1010101111111111111000001111100000001111001010101010110101010111'
# ls = '1010101111111111_1110000011111000_0000111100101010_1010110101010111'
key = '1000011101010101010101011011001010110101010111010100'
print(len(key))

ciphertext = Block_Cryptography(ls, key)
ciphertext

52


'0000000000000000111000001111100100001111001010100000000000000000'

In [255]:
x = '0111111111111111'
y = '1'
t = sum(x, y)
t

'1000000000000000'